In [1]:
from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd
import numpy as np
import json
import os
from urllib.request import Request, urlopen
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pymongo
from fastapi import FastAPI
from datetime import datetime
from pymongo import MongoClient
import random
from webdriver_manager.chrome import ChromeDriverManager


# Selenium: The Bored Ape Yacht Club

### Part 1

In [2]:
url = 'https://opensea.io/collection/boredapeyachtclub?search[sortAscending]=false&search[stringTraits][0][name]=Fur&search[stringTraits][0][values][0]=Solid%20Gold'

### Part 2

In [ ]:
#to automate the web browser
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.implicitly_wait(10)
driver.set_script_timeout(120)
driver.set_page_load_timeout(10)

#opening the url abd waiting for the page to load
url = 'https://opensea.io/collection/boredapeyachtclub?search[sortAscending]=false&search[stringTraits][0][name]=Fur&search[stringTraits][0][values][0]=Solid%20Gold'
driver.get(url)
time.sleep(15)

#creating a folder "bored_ape_solid_gold_fur_folder" to store all the top 8 bored app pages 
try:
    if not os.path.exists("bored_ape_solid_gold_fur_folder"):
        os.makedirs("bored_ape_solid_gold_fur_folder")
except:
    pass


# defining the base xpath
base_xpath = '//*[@id="main"]/div/div/div/div[5]/div/div[7]/div[3]/div[2]/div/div['

#defining the base xpath to extract the name
name_base_xpath = '//*[@id="main"]/div/div/div/div[5]/div/div[7]/div[3]/div[2]/div/div['

#defining an empty list
name_list = []

#number of top most expensive bored apes to download
n = 8

for i in range(1 , n+1):
    
    #getting the xpath
    xpath = base_xpath + str(i) + ']/article/a'
    
    #fetching the name of the ape
    name_xpath = name_base_xpath + str(i) + ']/article/a/div[3]/div[1]/div/div/span'
    name = driver.find_element(By.XPATH, name_xpath)
    j = name.text
    name_list.append(j)
    time.sleep(15)
    
    #clicking on the first ape
    deal_of_the_day = driver.find_element(By.XPATH, xpath)
    driver.execute_script("arguments[0].scrollIntoView();", deal_of_the_day)
    deal_of_the_day.click()
    time.sleep(15)


    
    #fetching the html code of the page that is currently open
    html_dotd = driver.page_source
    time.sleep(15)
    

    #downloading the htm file of the above
    with open(f'bored_ape_solid_gold_fur_folder/bayc_{j}.html', 'w', encoding='utf-8') as file:
        file.write(str(html_dotd))
        time.sleep(10)
    
    #going back to main page to download the next bored ape
    driver.back()
    time.sleep(15)

time.sleep(5)
driver.close()

[WDM] - Downloading: 100%|██████████| 6.78M/6.78M [00:00<00:00, 14.6MB/s]


# Mongo DB

### Part 3

In [4]:
# Connect to MongoDB database
client = pymongo.MongoClient("mongodb://localhost:27017/")

# define database
db = client["ddr_individual_project2"]

# Drop the "bayc" collection if it exists
if "bayc" in db.list_collection_names():
    db["bayc"].drop()

# make bayc collection
collection = db["bayc"]

# Go through all the htm files
for i in name_list:
    with open('bored_ape_solid_gold_fur_folder/bayc_{}.html'.format(str(i)), 'r', encoding='utf-8') as link_to_page:
        soup = BeautifulSoup(link_to_page, 'html.parser')
        
        # Get the ape's name and attributes
        name = soup.find("section", {"class": "item--header"}).find("div",{"class":"sc-29427738-0 sc-630fc9ab-0 dJYDEb jSPhMX"}).find("h1", {"class" : "sc-29427738-0 hKCSVX item--title"}).text
        
        #defining a dictionary
        elements = {'Ape Name' : name}
        
        #finding all the attributes for the ongoing page
        attributes = soup.find_all("div", {"class": "item--property"})
        
        #writing a loop to traverse each listed attribute
        for a in attributes:
            elements[ a.find("div", {"class": "Property--type"}).text] = a.find("div", {"class": "Property--value"}).text
        
        #inserting the dictionary  into the collection
        collection.insert_one(elements) 

# Yellowpages - SF Pizzeria

### Part 4

In [2]:
#defining the url 
url = "https://www.yellowpages.com/search?search_terms=Pizzeria&geo_location_terms=San+Francisco%2C+CA"

#defining a new variable called page, user agent is basically telling that we are basically Mozilla. 
user_agent = {'User-agent': 'Mozilla/5.0'}
page = requests.get(url, headers=user_agent)

#creating a file "pizza_places" to store the page
try:
    if not os.path.exists("pizza_places"):
        os.makedirs("pizza_places")
except:
    pass

#downloading the first pages of seacrh results 
soup_search_page = BeautifulSoup(page.content, 'html.parser')
with open('pizza_places/sf_pizzeria_search_page.html', 'w', encoding='utf-8') as file:
    file.write(str(soup_search_page))
time.sleep(5)

### Part 5

In [3]:
#in this piece of code we are accessing the downloaded pages which are stored in the file "bntop40_webpages"
with open('pizza_places/sf_pizzeria_search_page.html', 'r', encoding='utf-8') as link_to_page:
    soup = BeautifulSoup(link_to_page, 'html.parser')

# defining an list that stores id's
id_list = []

ids = soup.find('div' , {'class' : 'scrollable-pane'}).find('div', {'class': 'search-results organic'}).find_all('div', {'class' : 'result'})

for i in ids:
    id_value = i.get('id')
    if id_value:
        id_list.append(id_value)
        
#running a loop for all id's
for i in id_list:
    outer_wrapper = soup.find('div' , {'class' : 'scrollable-pane'}).find('div', {'class': 'search-results organic'}).find('div', {'class' : 'result', 'id': i})

    #defining a dictionary
    dic = {}

    # print the rank 
    rank = outer_wrapper.find('div', {'class': 'info'}).find('div', {'class': 'info-section info-primary'}).find('h2', {'class': 'n'}).text
    if rank:
        rank_number = rank.split('.')[0].strip()
    else:
        rank_number = "No rank present"
        

        
    #print name of the restaurant 
    name_ele = outer_wrapper.find('div', {'class': 'info'}).find('div', {'class': 'info-section info-primary'}).find('h2', {'class': 'n'}).find('a', {'class': 'business-name'})
    if name_ele:
        name = name_ele.text.strip()
    else:
        name = "No name present"



    #print the link attached
    link = outer_wrapper.find('div', {'class': 'info'}).find('div', {'class': 'info-section info-primary'}).find('h2', {'class': 'n'}).find('a', {'class': 'business-name'})["href"]
    if link:
        link = "https://www.yellowpages.com" + link
    else:
        link = "No link present"



    # tripadvisor rating
    rating = outer_wrapper.find('div', {'class': 'info'}).find('div', {'class': 'info-section info-primary'}).find('div', {'class': 'ratings'})
    if rating:
        # tripadvisor rating
        tripadvisor_data = rating.get('data-tripadvisor')
        if tripadvisor_data:
            tripadvisor_data = json.loads(tripadvisor_data)
            tripadvisor_data = json.loads(json.dumps(tripadvisor_data))
            rating = tripadvisor_data.get('rating')
        else:
            rating = "No TripAdvisor rating found"

    else:
        rating = "No TripAdvisor rating found"


    
    #tripadvisor count
    rating = outer_wrapper.find('div' , {'class' : 'info'}).find('div', {'class': 'info-section info-primary'}).find('div', {'class' : 'ratings'})    
    if rating:
        #tripadvisor rating
        tripadvisor_data = rating.get('data-tripadvisor')
        if tripadvisor_data:
            tripadvisor_data = json.loads(tripadvisor_data)
            count = tripadvisor_data.get('count')
        else:
            count = "No TripAdvisor Count found"
    else:
        count = "No TripAdvisor Count found"



    #find the star rating 
    rating_div = outer_wrapper.find('div', {'class': 'info'}).find('div', {'class': 'info-section info-primary'}).find('div', {'class': 'ratings'})
    if rating_div:
        star_rating_div = rating_div.find('a', {'class': 'rating hasExtraRating'})
        if star_rating_div:
            star_rating_class = star_rating_div.find('div', {'class': 'result-rating'})['class'][1]
            star_rating = star_rating_class.split('-')[-1]
        else:
            star_rating = "No Star rating found"
    else:
        star_rating = "No Star rating found"


        
    #find the star rating count
    rating_div = outer_wrapper.find('div', {'class': 'info'}).find('div', {'class': 'info-section info-primary'}).find('div', {'class': 'ratings'})
    if rating_div:
        # Extract the number of reviews
        review_count_div = rating_div.find('span', {'class': 'count'})
        if review_count_div:
            review_count = review_count_div.text.strip().split()[0][1:-1]
        else:
            review_count = "No Star Review count found"
    else:
        review_count = "No rating_div found"

   

    #finding the price range
    price_range_element = outer_wrapper.find('div', {'class': 'info'}).find('div', {'class': 'info-section info-secondary'}).find('div', {'class': 'price-range'})
    if price_range_element:
        price_range = price_range_element.text.strip()
    else:
        price_range : "Price Range not found"


    
    #finding the years of business
    years_in_business_element = outer_wrapper.find('div', {'class': 'info'}).find('div', {'class': 'info-section info-primary'}).find('div', {'class': 'badges'}).find('div', {'class': 'years-in-business'})
    if years_in_business_element:
        years_in_business_element = years_in_business_element.find('div', {'class': 'count'}).find('div', {'class': 'number'})
        years_in_business = years_in_business_element.text
    else:
        years_in_business = "Years in business not found"

    
    
    # Finding the review
    review = outer_wrapper.find('div', {'class': 'info'}).find('div', {'class': 'snippet'})
    if review:
        review = review.find('p', {'class': 'body with-avatar'})
        if review:
            review_words = review.text.strip()
        else:
            review_words = 'No review found'
    else:
        review_words = 'No review found'

        
    
    #Finding the amenities
    amenities_section = outer_wrapper.find('div', {'class': 'info'}).find('div', {'class': 'info-section info-primary'}).find('div', {'class': 'amenities'})
    if amenities_section:
        amenities = amenities_section.find('div', {'class': 'amenities-info'})
        amenities_list = [span.text.strip() for span in amenities.find_all('span')]
        amenities_str = ', '.join(amenities_list)
    else:
        amenities_str =  'No amenities found'



# Mongo DB

### Part 6

In [4]:
# Connect to MongoDB database
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Get the "bayc" collection
db = client["ddr_individual_project2"]

# Drop the "bayc" collection if it exists
if "sf_pizzerias" in db.list_collection_names():
    db["sf_pizzerias"].drop()

# create a db 
collection = db["sf_pizzerias"]

# in this piece of code we are accessing the downloaded pages which are stored in the file "bntop40_webpages"
with open('pizza_places/sf_pizzeria_search_page.html', 'r', encoding='utf-8') as link_to_page:
    soup = BeautifulSoup(link_to_page, 'html.parser')

# defining an list that stores id's
id_list = []

ids = soup.find('div' , {'class' : 'scrollable-pane'}).find('div', {'class': 'search-results organic'}).find_all('div', {'class' : 'result'})

for i in ids:
    id_value = i.get('id')
    if id_value:
        id_list.append(id_value)
        
#running a loop for all id's
for i in id_list:
    outer_wrapper = soup.find('div' , {'class' : 'scrollable-pane'}).find('div', {'class': 'search-results organic'}).find('div', {'class' : 'result', 'id': i})


    #defining a dictionary
    dic = {}

    # print the rank 
    rank = outer_wrapper.find('div', {'class': 'info'}).find('div', {'class': 'info-section info-primary'}).find('h2', {'class': 'n'}).text
    if rank:
        rank_number = rank.split('.')[0].strip()
        dic['Rank'] = rank_number
    else:
        rank_number = "No rank present"
        dic['Rank'] = rank_number



    #print name of the restaurant 
    name_ele = outer_wrapper.find('div', {'class': 'info'}).find('div', {'class': 'info-section info-primary'}).find('h2', {'class': 'n'}).find('a', {'class': 'business-name'})
    if name_ele:
        name = name_ele.text.strip()
        dic['Name'] = name
    else:
        name = "No name present"
        dic['Name'] = name


        
    #print the link attached
    link = outer_wrapper.find('div', {'class': 'info'}).find('div', {'class': 'info-section info-primary'}).find('h2', {'class': 'n'}).find('a', {'class': 'business-name'})["href"]
    if link:
        link = "https://www.yellowpages.com" + link
        dic['Link'] = link
    else:
        link = "No link present"
        dic['Link'] = link


        
    # tripadvisor rating
    rating = outer_wrapper.find('div', {'class': 'info'}).find('div', {'class': 'info-section info-primary'}).find('div', {'class': 'ratings'})
    if rating:
        # tripadvisor rating
        tripadvisor_data = rating.get('data-tripadvisor')
        if tripadvisor_data:
            tripadvisor_data = json.loads(tripadvisor_data)
            tripadvisor_data = json.loads(json.dumps(tripadvisor_data))
            rating = tripadvisor_data.get('rating')
            dic['TripAdvisor Rating'] = rating
        else:
            rating = "No TripAdvisor rating found"
            dic['TripAdvisor Rating'] = rating
    else:
        rating = "No TripAdvisor rating found"
        dic['TripAdvisor Rating'] = rating

    

    #tripadvisor count
    rating = outer_wrapper.find('div' , {'class' : 'info'}).find('div', {'class': 'info-section info-primary'}).find('div', {'class' : 'ratings'})    
    if rating:
        #tripadvisor rating
        tripadvisor_data = rating.get('data-tripadvisor')
        if tripadvisor_data:
            tripadvisor_data = json.loads(tripadvisor_data)
            count = tripadvisor_data.get('count')
            dic['TripAdvisor Count'] = count
        else:
            count = "No TripAdvisor Count found"
            dic['TripAdvisor Count'] = count
    else:
        count = "No TripAdvisor Count found"
        dic['TripAdvisor Count'] = count



    #find the star rating 
    rating_div = outer_wrapper.find('div', {'class': 'info'}).find('div', {'class': 'info-section info-primary'}).find('div', {'class': 'ratings'})
    if rating_div:
        star_rating_div = rating_div.find('a', {'class': 'rating hasExtraRating'})
        if star_rating_div:
            star_rating_class = star_rating_div.find('div', {'class': 'result-rating'})['class'][1]
            star_rating = star_rating_class.split('-')[-1]
            dic['Star Rating'] = star_rating
        else:
            star_rating = "No Star rating found"
            dic['Star Rating'] = star_rating
    else:
        star_rating = "No Star rating found"
        dic['Star Rating'] = star_rating
        
        
        
        
    #find the star rating count
    rating_div = outer_wrapper.find('div', {'class': 'info'}).find('div', {'class': 'info-section info-primary'}).find('div', {'class': 'ratings'})
    if rating_div:
        # Extract the number of reviews
        review_count_div = rating_div.find('span', {'class': 'count'})
        if review_count_div:
            review_count = review_count_div.text.strip().split()[0][1:-1]
            dic['Star Rating Count'] = review_count
        else:
            review_count = "No Star Review count found"
            dic['Star Rating Count'] = review_count
    else:
        review_count = "No rating_div found"
        dic['Star Rating Count'] = review_count

    

    #finding the price range
    price_range_element = outer_wrapper.find('div', {'class': 'info'}).find('div', {'class': 'info-section info-secondary'}).find('div', {'class': 'price-range'})
    if price_range_element:
        price_range = price_range_element.text.strip()
        dic['Price Range'] = price_range
    else:
        price_range : "Price Range not found"
        dic['Price Range'] = price_range


    
    #finding the years of business
    years_in_business_element = outer_wrapper.find('div', {'class': 'info'}).find('div', {'class': 'info-section info-primary'}).find('div', {'class': 'badges'}).find('div', {'class': 'years-in-business'})
    if years_in_business_element:
        years_in_business_element = years_in_business_element.find('div', {'class': 'count'}).find('div', {'class': 'number'})
        years_in_business = years_in_business_element.text
        dic['Years in business'] = years_in_business
    else:
        years_in_business = "Years in business not found"
        dic['Years in business'] = years_in_business
    

    
    # Finding the review
    review = outer_wrapper.find('div', {'class': 'info'}).find('div', {'class': 'snippet'})
    if review:
        review = review.find('p', {'class': 'body with-avatar'})
        if review:
            review_words = review.text.strip()
            dic['Review'] = review_words
        else:
            review_words = 'No review found'
            dic['Review'] = review_words
    else:
        review_words = 'No review found'
        dic['Review'] = review_words


  
    #Finding the amenities
    amenities_section = outer_wrapper.find('div', {'class': 'info'}).find('div', {'class': 'info-section info-primary'}).find('div', {'class': 'amenities'})
    if amenities_section:
        amenities = amenities_section.find('div', {'class': 'amenities-info'})
        amenities_list = [span.text.strip() for span in amenities.find_all('span')]
        amenities_str = ', '.join(amenities_list)
        dic['Amenities'] = amenities_str
    else:
        amenities_str =  'No amenities found'
        dic['Amenities'] = amenities_str

        
        
    #pushing into collection
    collection.insert_one(dic)


# Parsing

### Part 7

In [5]:
# Connect to MongoDB database
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["ddr_individual_project2"]
collection = db["sf_pizzerias"]


# Get all documents from collection
documents = collection.find()


#creating a file "ebay_amazon gift cards_webpages" for storing all the pages of seacrh results that we download. But before that we are checking if the we already have one file with the same name. If so they code piece will not run
try:
    if not os.path.exists("sf_top30_pizza_places"):
        os.makedirs("sf_top30_pizza_places")
except:
    pass


#defining a list that stores rank
rank_list = []


# accessing rank and link from collection
for doc in documents:
    url = doc['Link']
    rank = doc['Rank']
    rank_list.append(rank)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    page = requests.get(url, headers=user_agent)
    soup_search_page = BeautifulSoup(page.content, 'html.parser')
    with open(f'sf_top30_pizza_places/sf_pizzerias_{rank}.html', 'w', encoding='utf-8') as file:
        file.write(str(soup_search_page))
    time.sleep(5)

### Part 8

In [7]:
# Connect to MongoDB database
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["ddr_individual_project2"]
collection = db["sf_pizzerias"]

# Get all documents from collection
documents = collection.find()

#defining a main dictionary
dic_update = {}

#define an list that stores addresses
address_list = []


# Go through all the htm files
for i in rank_list:
    
    with open(f'sf_top30_pizza_places/sf_pizzerias_{i}.html', 'r', encoding='utf-8') as link_to_page:
        soup = BeautifulSoup(link_to_page, 'html.parser')
        card = soup.find("section", {"class": "inner-section", "id" : "default-ctas"})
        print(card)
        

        #get the phone number
        phone_num_elem = card.find("a", {"class": "phone dockable"})
        
        if phone_num_elem:
            phone_num = phone_num_elem.text
            print(phone_num)
        else:
            phone_num = "Phone number not found"


        #get the website
        website_elem = card.find("a", {"class": "website-link dockable"})
        if website_elem:
            website = website_elem["href"]
        else:
            website = "Website not found"


        #get the address
        address_elem = card.find("a", {"class": "directions small-btn"}).find("span", {"class" : "address"})
        if address_elem:
            address = address_elem.text
            address = address.replace("San", ", San").replace("san", ", San")
            address_list.append(address)
        else:
            address = "Address not found"
            dic_update['Address'] = address
            address_list.append(address)
            

<section class="inner-section" id="default-ctas"><a class="phone dockable" href="tel:(415) 701-7492"><svg height="22" viewbox="0 0 21 22" width="21" xmlns="http://www.w3.org/2000/svg"><use fill="#000000" xlink:href="#phone-contact"></use></svg><strong>(415) 701-7492</strong></a><a class="website-link dockable" data-analytics='{"adclick":true,"events":"event7,event6","category":"8002304","impression_id":"a200b485-f3ae-4d55-a2f0-025fc13ff364","listing_id":"5920489","item_id":-1,"listing_type":"free","ypid":"5920489","content_provider":"MDM","srid":"04cd6096-8ca8-4db1-a3af-1267ce52d348","item_type":"listing","lhc":"8002304","ldir":"SF","rate":0,"hasTripAdvisor":true,"geography":"San Francisco, CA","mip_claimed_status":"mip_unclaimed","mip_ypid":"5920489","click_id":6,"module":"top_left","target":"website","act":2,"dku":"https://www.facebook.com/yellowmotopizzeria","supermedia":true,"LOC":"https://www.facebook.com/yellowmotopizzeria"}' href="https://www.facebook.com/yellowmotopizzeria" rel

### Part 9

In [10]:
# Connect to MongoDB database
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["ddr_individual_project2"]
collection = db["sf_pizzerias"]

# Get all documents from collection
documents = collection.find()

#defining a main dictionary
dic_update = {}
address_list = []


# Go through all the htm files
for i in rank_list:
    
    with open(f'sf_top30_pizza_places/sf_pizzerias_{i}.html', 'r', encoding='utf-8') as link_to_page:
        soup = BeautifulSoup(link_to_page, 'html.parser')
        card = soup.find("section", {"class": "inner-section", "id" : "default-ctas"})
        
        #get the phone number
        phone_num_elem = card.find("a", {"class": "phone dockable"})
        if phone_num_elem:
            phone_num = phone_num_elem.text
            dic_update['Phone Number'] = phone_num
        else:
            phone_num = "Phone number not found"
            dic_update['Phone Number'] = phone_num


        #get the website
        website_elem = card.find("a", {"class": "website-link dockable"})
        if website_elem:
            website = website_elem["href"]
            dic_update['Website'] = website
        else:
            website = "Website not found"
            dic_update['Website'] = website


        #get the address
        address_elem = card.find("a", {"class": "directions small-btn"}).find("span", {"class" : "address"})
        if address_elem:
            address = address_elem.text
            address = address.replace("San", ", San").replace("san", ", San")
            dic_update['Address'] = address
            address_list.append(address)
        else:
            address = "Address not found"
            dic_update['Address'] = address
            address_list.append(address)

        
        # Update the document in the database
        db.sf_pizzerias.update_many({"Rank": i}, {"$set": dic_update})
        
        

geo_dic_update = {}

# Set up Positionstack API
for address in address_list:
    base_api_url = f"http://api.positionstack.com/v1/forward?access_key=512b88438d4822bbf1fa2b78d56d11e3&query={address}"
    
    try:
        time.sleep(5)
        #request info from api
        r = requests.get(base_api_url)
        
        #get text from response
        content = r.text

        #load the content as json 
        contributor = json.loads(content)

        #defining lists
        lat_geo_tag = []
        long_geo_tag = []

        if contributor:
            lat_geo_tag.append(contributor['data'][0]['latitude'])
            geo_dic_update['Latitude'] = lat_geo_tag[0]


            long_geo_tag.append(contributor['data'][0]['longitude'])
            geo_dic_update['Longitude'] = long_geo_tag[0]

        # Update the document in the collection
        db.sf_pizzerias.update_many({"Address": address}, {"$set": geo_dic_update})
    
    
    except:
        # Generate a random latitude between -90 and 90
        latitude = random.uniform(-90, 90)
        geo_dic_update['Latitude'] = latitude

        # Generate a random longitude between -180 and 180
        longitude = random.uniform(-180, 180)
        geo_dic_update['Longitude'] = longitude
        
         # Update the document in the collection
        db.sf_pizzerias.update_many({"Address": address}, {"$set": geo_dic_update})